In [29]:
import time
import pandas as pd
import numpy as np
import streamlit as st
from openai._client import OpenAI
import json

client = OpenAI(
    api_key=st.secrets["openai"]["api_key"],
)

In [30]:
df_tags = pd.read_csv("./data/df_tags_use_app_22_11.csv")
print(len(df_tags))
df_tags = df_tags.drop_duplicates(subset=['url'])
print(len(df_tags))

291
277


In [31]:
# load the dict from pickle file
import pickle
with open('./data/url_technical_text_dict.pkl', 'rb') as handle:
    url_text_dict = pickle.load(handle)

In [32]:


def generate_tags_and_handle_rate_limit(df_tags):
    df_tags["is_software"] = ""

    # Iterate over the rows of the dataframe
    for index, row in df_tags.iterrows():
        print(index)
        # Create a prompt using the Product_Name, category, and the text from the website
        url = row["url"]
        product_name = row["Product_Name"]
        product_category = row["Product category"]

        website_text = url_text_dict.get(url, "")

        prompt = f"Given the website text: '{website_text}', and the product name: '{product_name}', is the product a software or a hardware? If it is a combination, then it is hardware. If its only software, then return software.  Return true if it's a software or false if it's a hardware."

        # Call the general_gpt function with this prompt
        for _ in range(3):  # Retry up to 3 times
            try:
                response = client.chat.completions.create(
                    model="gpt-4-1106-preview",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant that determines whether a product is software or hardware. Your responses should be in JSON format and consist of a boolean value on the format: {'is_software': true/false} "}, 
                        {"role": "user", "content": "{}".format(prompt)}
                    ],
                    max_tokens=50,
                    response_format={ "type": "json_object" },
                    timeout=10  # Add a timeout
                )
                is_software = response.choices[0].message.content.strip()
                print(product_name)
                print(is_software)
                is_software_dict = json.loads(is_software)
                # Access the value of the 'is_software' key
                is_software_value = is_software_dict['is_software']
                print(is_software_value)
                print("_________________________________")
                # Save the determination in the 'is_software' column
                df_tags.loc[index, "is_software"] = is_software_value
                break
            except Exception as e:
                print(f"Error at index {index}: {e}")
                time.sleep(5)  # Wait for 5 seconds before retrying
            else:
                print(f"Skipping index {index} after 3 failed attempts")
                continue  # Skip the current index if the API call fails 3 times

    return df_tags

#read csv to df_tags

df_tags_tech = generate_tags_and_handle_rate_limit(df_tags)
df_tags_tech.to_csv("./data/df_tags_use_app_22_11_issoftware.csv", index=False)

0
Product Name
{
    "is_software": false
}
False
_________________________________
1
K-Sync Synchronization unit
{"is_software": false}
False
_________________________________
2
cNODE Midi - Transponder
{"is_software": false}
False
_________________________________
3
cNODE - Transponder, Embedable
{"is_software": false}
False
_________________________________
4
cNODE MiniS - Transponder
{"is_software": false}
False
_________________________________
5
cNODE Maxi - Transponder
{"is_software": false}
False
_________________________________
6
cNODE Micro - Transponder
{"is_software": false}
False
_________________________________
7
cNODE IQAM - Intelligent data analysis and monitoring
{"is_software": false}
False
_________________________________
8
cNODE - Transponder, for explosive atmosphere
{"is_software": false}
False
_________________________________
9
TTC 30 & TTC 10 - Transponders test and configuration units
{"is_software": false}
False
_________________________________
10
cNODE M

In [33]:

df_tags_tech.to_excel("./data/df_tags_use_app_22_11_issoftware.xlsx", index=False)

In [35]:
df = pd.read_csv("data/df_tags_use_app_22_11_issoftware.csv")
df.to_excel("data/df_tags_use_app_22_11_issoftware.xlsx")

In [42]:
df_new = pd.read_excel("data/df_tags_use_app_22_11_issoftware.xlsx")
df_new.to_csv("data/df_tags_use_app_22_11_issoftware.csv", index=False)